# Running Simulations Directly in Python

This notebook shows how to run simulations directly in Python without the API.

## Import Required Modules

In [ ]:
from cascabel.models.waitline import WaitLine
from cascabel.models.simulation import Simulation
from cascabel.models.models import BorderCrossingConfig, SimulationConfig, PhoneConfig
import matplotlib.pyplot as plt
import pandas as pd

## Create a WaitLine

The WaitLine represents the geographic path that cars follow.

In [ ]:
# Create waitline from GeoJSON
waitline = WaitLine(
    geojson_path="cascabel/paths/usa2mx/bota.geojson",
    speed_regime={"slow": 0.8, "fast": 0.2},
    line_length_seed=0.5
)

print(f"WaitLine created with {len(waitline.path)} points")
print(f"Total length: {waitline.total_length:.2f} meters")

## Configure the Border Crossing

Set up queues, arrival rates, service rates, etc.

In [ ]:
# Configure border crossing
border_config = BorderCrossingConfig(
    num_queues=2,  # 2 lanes
    nodes_per_queue=[1, 1],  # 1 booth per lane
    arrival_rate=1.0,  # 1 car per minute
    service_rates=[0.8, 0.9],  # service rates for each booth
    queue_assignment='shortest',  # assign to shortest queue
    safe_distance=8.0,  # meters between cars
    max_queue_length=50
)

print("Border config:")
print(f"- {border_config.num_queues} queues")
print(f"- Arrival rate: {border_config.arrival_rate} cars/min")
print(f"- Service rates: {border_config.service_rates}")

## Configure the Simulation

In [ ]:
# Simulation configuration
simulation_config = SimulationConfig(
    max_simulation_time=1800.0,  # 30 minutes
    time_factor=1.0,
    enable_telemetry=True,
    enable_position_tracking=True
)

# Phone configuration for telemetry
phone_config = PhoneConfig(
    sampling_rate=5.0,  # Hz
    accelerometer_noise=0.01,
    gyroscope_noise=0.001,
    gps_accuracy=5.0  # meters
)

## Run the Simulation

In [ ]:
# Create simulation
simulation = Simulation(
    waitline=waitline,
    border_config=border_config,
    simulation_config=simulation_config,
    phone_config=phone_config
)

# Run simulation
print("Running simulation...")
simulation()
print("Simulation completed!")

## Get Results

In [ ]:
# Get statistics
stats = simulation.get_statistics()

print("\n=== Simulation Results ===")
print(f"Duration: {stats.simulation_duration:.0f} seconds")
print(f"Total arrivals: {stats.execution_stats.total_arrivals}")
print(f"Total completions: {stats.execution_stats.total_completions}")
print(f"Average queue length: {stats.execution_stats.average_queue_length:.2f}")
print(f"Average wait time: {stats.execution_stats.average_wait_time:.2f} seconds")
print(f"Utilization: {stats.execution_stats.overall_utilization:.1%}")